In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time



driver = webdriver.Chrome()

###### 스파르타 크롤링

# 검색어 입력 및 검색

fast_campus_search_url = "https://spartacodingclub.kr/catalog"

driver.get(fast_campus_search_url)
time.sleep(0.1)
# CLASS_NAME을 이용하여 검색버튼 클릭
elements=driver.find_elements(By.CLASS_NAME,'css-1ef4nm4')
elements[0].click()
# input 데이터 입력
input_element = driver.find_element(By.XPATH, '//*[@id="__next"]/div[5]/div/div[1]/div[1]/div/div[1]/input')

# input 요소에 데이터 입력 후 엔터 키 누르기
input_element.send_keys("데이터")
input_element.send_keys(Keys.ENTER)
list1=[]
for i in range(1,99):
    try:
        title=driver.find_element(By.XPATH, f'//*[@id="__next"]/div[5]/div/div[1]/div[2]/div/div[{i}]/div[2]/div[1]').text
        list1.append(title.split('\n')[0])
    except:
        continue
# CLASS_NAME을 이용하여 검색 닫기버튼 클릭
elements=driver.find_elements(By.CLASS_NAME,'css-2s8b32')
elements[0].click()     

time.sleep(0.1)

# CLASS_NAME을 이용하여 검색버튼 클릭
elements=driver.find_elements(By.CLASS_NAME,'css-1ef4nm4')
elements[0].click()

# input 데이터 입력
input_element = driver.find_element(By.XPATH, '//*[@id="__next"]/div[5]/div/div[1]/div[1]/div/div[1]/input')

# input 요소에 데이터 입력 후 엔터 키 누르기
input_element.send_keys("data")
input_element.send_keys(Keys.ENTER)
for i in range(1,99):
    try:
        title=driver.find_element(By.XPATH, f'//*[@id="__next"]/div[5]/div/div[1]/div[2]/div/div[{i}]/div[2]/div[1]').text
        list1.append(title.split('\n')[0])
    except:
        continue
#list1의 중복값을 제거하여 list2에 저장
list2=[]
for i in list1:
    if i not in list2:
        list2.append(i)
df=pd.DataFrame(list2)
df.to_csv('Sparta.csv',index=False)

###### 인프런 크롤링

keyword = "데이터"
url = f"https://www.inflearn.com/courses?s={keyword}"

driver.get(url)
ind = 1  # 현재 복사한 순서
no = 1  # 1~5페이지 중 위치한 곳
list1 = []  # 결과물이 저장되는 리스트

while 1:
    try:
        time.sleep(0.1)
        # 제목명
        title = driver.find_element(by='xpath',value = f'//*[@id="courses_section"]/div/div/div/main/div[4]/div/div[{ind}]/div/a/div[2]/div[1]').text    
        list1.append([title])
        ind += 1
        
    except:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        # 더보기 버튼 찾기
        if no == 1:
            driver.find_element(by='xpath',value = '//*[@id="courses_section"]/div/div/div/footer/nav/div/a').click()
            no += 1
            ind = 1
            continue
           
        elif no < 52:
            driver.find_element(by='xpath',value = '//*[@id="courses_section"]/div/div/div/footer/nav/div/a[2]').click()
            no += 1
            ind = 1
            continue
        
        # 더이상 이동할수 없을 경우 종료
        else:
            break
            
data_ko = pd.DataFrame(list1)
#data_ko.shape

#데이터로 검색하여 가져오기 + 영어(page 8)
# 검색어 입력 및 검색
keyword = "data"
url = f"https://www.inflearn.com/courses?s={keyword}"

driver.get(url)

ind = 1  # 현재 복사한 순서
no = 1  # 1~5페이지 중 위치한 곳
list1 = []  # 결과물이 저장되는 리스트

while 1:
    try:
        time.sleep(0.1)
        # 제목명
        title = driver.find_element(by='xpath',value = f'//*[@id="courses_section"]/div/div/div/main/div[4]/div/div[{ind}]/div/a/div[2]/div[1]').text    
        list1.append([title])
        ind += 1
        
    except:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        # 더보기 버튼 찾기
        if no == 1:
            driver.find_element(by='xpath',value = '//*[@id="courses_section"]/div/div/div/footer/nav/div/a').click()
            no += 1
            ind = 1
            continue
           
        if no < 8:
            driver.find_element(by='xpath',value = '//*[@id="courses_section"]/div/div/div/footer/nav/div/a[2]').click()
            no += 1
            ind = 1
            continue
        
        # 더이상 이동할수 없을 경우 종료
        else:
            break
data_en = pd.DataFrame(list1)
#data_en.shape

#data_ko과 data_en 합치기
data_add = pd.concat([data_ko,data_en])

# 중복 데이터 삭제
data_del = data_add.drop_duplicates([0])

#저장 전 columns 명 'title'로 변경
data_del.columns = ['title']

#데이터 Crawling_inflearn.csv로 저장(UTF-8)
data_del.to_csv("./Inflearn.csv",index=False)


#### 패스트캠퍼스 크롤링

#selenium사용을 위한 초반작업
keyword = "데이터"
fastcampus_url= f"https://fastcampus.co.kr/search?keyword={keyword}"
driver.get(fastcampus_url)

# 한글로 "데이터"
# 페이지의 높이를 얻어오기
page_height = driver.execute_script("return document.body.scrollHeight")

# 스크롤을 맨 아래까지 내리기
while True:
    # 스크롤을 현재 페이지 높이만큼 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 페이지 로딩 대기
    time.sleep(1)
    
    # 스크롤을 다 내렸을 때 루프 종료
    new_page_height = driver.execute_script("return document.body.scrollHeight")
    if new_page_height == page_height:
        break
    page_height = new_page_height
    
    
# 맨 위로 스크롤 올리기
driver.execute_script("window.scrollTo(0, 0);")
###스크롤 내리는 이유:동적웹페이지로 모든 정보를 크롤링하기 위해




#데이터에 대한 "count개의 검색결과" 에서 "count개" 를 추출하는 코드
count = driver.find_elements(By.XPATH,'//*[@id="main"]/div/section/div/p/span[2]')[0].text
count=''.join(filter(str.isdigit,count))
count = int(count)


#강좌제목 크롤링:전자책의 경우 XPATH가 달라서 예외처리로 크롤링
title_list = []
for i in range(1,count+1):
    try:
        title =driver.find_elements(By.XPATH,f'//*[@id="main"]/div/section/div/div[2]/div[{i}]/div[3]/strong')[0].text
        title_list.append(title)
    except:
        title_book = driver.find_elements(By.XPATH,f'//*[@id="main"]/div/section/div/div[2]/div[{i}]/div[4]/strong')[0].text
        title_list.append(title_book)



#영어로 data
keyword = "data"
fastcampus_url= f"https://fastcampus.co.kr/search?keyword={keyword}"

driver.get(fastcampus_url)

# 페이지의 높이를 얻어오기
page_height = driver.execute_script("return document.body.scrollHeight")

# 스크롤을 맨 아래까지 내리기
while True:
    # 스크롤을 현재 페이지 높이만큼 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 페이지 로딩 대기
    time.sleep(1)
    
    # 스크롤을 다 내렸을 때 루프 종료
    new_page_height = driver.execute_script("return document.body.scrollHeight")
    if new_page_height == page_height:
        break
    page_height = new_page_height
    
    
# 맨 위로 스크롤 올리기
driver.execute_script("window.scrollTo(0, 0);")
###스크롤 내리는 이유:동적웹페이지로 모든 정보를 크롤링하기 위해




#데이터에 대한 count개의 검색결과 에서 count개 를 추출하는 코드
count = driver.find_elements(By.XPATH,'//*[@id="main"]/div/section/div/p/span[2]')[0].text
count=''.join(filter(str.isdigit,count))
count = int(count)


#강좌제목 크롤링:전자책의 경우 XPATH가 달라서 예외처리로 크롤링
title_list2=[]
for i in range(1,count+1):
    try:
        title =driver.find_elements(By.XPATH,f'//*[@id="main"]/div/section/div/div[2]/div[{i}]/div[3]/strong')[0].text
        title_list2.append(title)
    except:
        title_book = driver.find_elements(By.XPATH,f'//*[@id="main"]/div/section/div/div[2]/div[{i}]/div[4]/strong')[0].text
        title_list2.append(title_book)


#한글데이터,영어data 합침
fastcampus_title = title_list + title_list2

# 데이터프레임으로 변환
df = pd.DataFrame({'강좌명': fastcampus_title})
#혹시 중복이 있다면 제거
df = df.drop_duplicates()
df.to_csv("Fastcamp.csv",index=False)

